In [1]:
%%configure
{ "conf": {
            "spark.jars":"hdfs:///apps/hudi/lib/hudi-spark-bundle.jar,hdfs:///apps/hudi/lib/spark-avro.jar",
            "spark.serializer":"org.apache.spark.serializer.KryoSerializer",
            "spark.sql.hive.convertMetastoreParquet":"false"
          }}

In [2]:
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.functions._
import org.apache.hudi.DataSourceWriteOptions
import org.apache.hudi.config.HoodieWriteConfig
import org.apache.hudi.hive.MultiPartKeysValueExtractor

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1605639433408_0007,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
No module named 'org'
Traceback (most recent call last):
ModuleNotFoundError: No module named 'org'



In [3]:
snapshotQueryDF = spark.read.format('org.apache.hudi').load('s3://md-labs-hudi-demo-data-bucket/hudi/retail_transactions' + '/*/*')
    
snapshotQueryDF.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------+------------------+----------------------+--------------------+--------------------+-------+----------+--------+-------------+-----------+---------+--------+------+---+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|     dms_received_ts|tran_id| tran_date|store_id|   store_city|store_state|item_code|quantity| total| Op|
+-------------------+--------------------+------------------+----------------------+--------------------+--------------------+-------+----------+--------+-------------+-----------+---------+--------+------+---+
|     20201117201857|  20201117201857_0_1|                15|            2019-03-16|c85559e3-8ba7-4ec...|2020-11-17 20:17:...|     15|2019-03-16|       7|      CHICAGO|         IL|   XXXXXX|      17|126.25|  I|
|     20201117201857|  20201117201857_1_2|                 1|            2019-03-17|e8423d98-be0e-4a7...|2020-11-17 20:17:...|      1|2019-03-17|       1|  

In [4]:
readOptions = {
  'hoodie.datasource.query.type': 'incremental',
  'hoodie.datasource.read.begin.instanttime': 20201117201857,
}

incQueryDF = spark.read.format('org.apache.hudi').options(**readOptions).load('s3://md-labs-hudi-demo-data-bucket/hudi/retail_transactions')
    
incQueryDF.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------+------------------+----------------------+--------------------+--------------------+-------+----------+--------+----------+-----------+---------+--------+------+---+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|     dms_received_ts|tran_id| tran_date|store_id|store_city|store_state|item_code|quantity| total| Op|
+-------------------+--------------------+------------------+----------------------+--------------------+--------------------+-------+----------+--------+----------+-----------+---------+--------+------+---+
|     20201117235041|  20201117235041_0_2|                 7|            2019-03-14|59549b05-8baf-4ac...|2020-11-17 23:47:...|      7|2019-03-14|       2|  NEW YORK|         NY|   XXXXXX|      11|166.25|  I|
|     20201117235041|  20201117235041_1_1|                 6|            2019-03-18|8f1af2ad-32a4-44f...|2020-11-17 23:47:...|      6|2019-03-18|       1|   CHICAGO|   